In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import os

In [2]:
import wandb
wandb.init(project="breast-cancer-classifier", name="resnet-run1")

wandb: Currently logged in as: canatilgan (canatilgan-lule-university-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [3]:
from torchvision.datasets import ImageFolder

class IgnoreCheckpointsFolder(ImageFolder):
    def find_classes(self, directory):
        classes = [d.name for d in os.scandir(directory) if d.is_dir() and d.name != ".ipynb_checkpoints"]
        classes.sort()
        class_to_idx = {cls_name: i for i, cls_name in enumerate(classes)}
        return classes, class_to_idx


In [4]:
data_dir = "Dataset_2_breast_cancer_histopathology_400X" 
img_size = 224
batch_size = 32

transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])
])

train_dataset = IgnoreCheckpointsFolder(os.path.join(data_dir, 'train'), transform=transform)
test_dataset = IgnoreCheckpointsFolder(os.path.join(data_dir, 'test'), transform=transform)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("Classes:", train_dataset.classes)


Classes: ['benign', 'malignant']


In [5]:
##1st Option##
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),

            nn.Flatten(),
            nn.Linear(32 * 56 * 56, 128),
            nn.ReLU(),
            nn.Linear(128, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.model(x)

model = SimpleCNN()

In [6]:
##2nd Option##
from torchvision.models import resnet18, ResNet18_Weights
weights = ResNet18_Weights.DEFAULT
model = resnet18(weights=weights)

model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 1),
    nn.Sigmoid()
)

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.unsqueeze(1).float()  # Ensure shape [batch, 1]

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

    train_loss = running_loss / len(train_loader)
    train_accuracy = 100 * correct / total

    print(f"Epoch {epoch+1}/{epochs} - Train loss: {train_loss:.4f}, Train acc: {train_accuracy:.2f}%")

    #✅ Log metrics to W&B
    wandb.log({"epoch": epoch + 1, "loss": train_loss, "accuracy": train_accuracy})

Epoch 1/10 - Train loss: 0.4024, Train acc: 82.61%
Epoch 2/10 - Train loss: 0.2528, Train acc: 90.52%
Epoch 3/10 - Train loss: 0.1889, Train acc: 93.30%
Epoch 4/10 - Train loss: 0.1803, Train acc: 92.96%
Epoch 5/10 - Train loss: 0.1213, Train acc: 95.57%
Epoch 6/10 - Train loss: 0.1308, Train acc: 95.39%
Epoch 7/10 - Train loss: 0.1325, Train acc: 94.52%
Epoch 8/10 - Train loss: 0.1321, Train acc: 94.96%
Epoch 9/10 - Train loss: 0.0682, Train acc: 97.83%
Epoch 10/10 - Train loss: 0.1089, Train acc: 96.00%


In [8]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in test_loader:
        inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
        outputs = model(inputs)
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.size(0)

test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

wandb.log({
    "test_accuracy": test_accuracy,
})

Test Accuracy: 92.11%
